In [1]:
import pickle
import pandas as pd
import numpy as np
import datetime  
import pickle
from math import floor
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDRegressor, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sqlalchemy import create_engine
from sqlalchemy import text
import csv

Inserting data base information and connecting to it

In [2]:
URI = ""
PORT = ""
DB = ""
USER = "admin"
PASSWORD = ""   
engine = create_engine("mysql+mysqldb://{}:{}@{}:{}/{}".format(USER, PASSWORD, URI, PORT, DB), echo=True)

In [3]:
"""
# Get the data
availability_data = engine.execute("SELECT * FROM availability2;")

# Create CSV file
filename = "bike_data.csv"

# Open the CSV file in write mode
with open(filename, mode='w', newline='') as file:

    # Create a CSV writer object
    writer = csv.writer(file)

    # Write the header row (optional)
    writer.writerow(['number', 'available_bikes', 'available_bike_stands', 'status', 'datetime'])

    # Write the data rows
    for row in availability_data:
        writer.writerow(row)
"""

'\n# Get the data\navailability_data = engine.execute("SELECT * FROM availability2;")\n\n# Create CSV file\nfilename = "bike_data.csv"\n\n# Open the CSV file in write mode\nwith open(filename, mode=\'w\', newline=\'\') as file:\n\n    # Create a CSV writer object\n    writer = csv.writer(file)\n\n    # Write the header row (optional)\n    writer.writerow([\'number\', \'available_bikes\', \'available_bike_stands\', \'status\', \'datetime\'])\n\n    # Write the data rows\n    for row in availability_data:\n        writer.writerow(row)\n'

In [4]:
dfa = pd.read_csv('bike_data.csv')
dfa.head()

,number,available_bikes,available_bike_stands,status,datetime
0,42,8,22,OPEN,1677760001
1,30,2,18,OPEN,1677760001
2,54,14,18,OPEN,1677760001
3,108,18,17,OPEN,1677760001
4,20,22,8,OPEN,1677760001


In [5]:
"""
# Get the data
weather_data = engine.execute("SELECT * FROM weather2;")

# Create CSV file
filename = "weather_data.csv"

# Open the CSV file in write mode
with open(filename, mode='w', newline='') as file:

    # Create a CSV writer object
    writer = csv.writer(file)

    # Write the header row (optional)
    writer.writerow(['main', 'description', 'temp', 'visibility', 'wind_speed', 'wind_direction', 'clouds', 'datetime'])

    # Write the data rows
    for row in weather_data:
        writer.writerow(row)
"""

'\n# Get the data\nweather_data = engine.execute("SELECT * FROM weather2;")\n\n# Create CSV file\nfilename = "weather_data.csv"\n\n# Open the CSV file in write mode\nwith open(filename, mode=\'w\', newline=\'\') as file:\n\n    # Create a CSV writer object\n    writer = csv.writer(file)\n\n    # Write the header row (optional)\n    writer.writerow([\'main\', \'description\', \'temp\', \'visibility\', \'wind_speed\', \'wind_direction\', \'clouds\', \'datetime\'])\n\n    # Write the data rows\n    for row in weather_data:\n        writer.writerow(row)\n'

In [6]:
dfw = pd.read_csv('weather_data.csv')
dfw.head()

,main,description,temp,visibility,wind_speed,wind_direction,clouds,datetime
0,Clouds,broken clouds,280.72,10000.0,4.12,70.0,75.0,1677760002
1,Clouds,broken clouds,280.72,10000.0,4.12,70.0,75.0,1677760064
2,Clouds,broken clouds,280.72,10000.0,4.12,70.0,75.0,1677760366
3,Clouds,broken clouds,280.95,10000.0,4.12,70.0,75.0,1677760668
4,Clouds,broken clouds,280.95,10000.0,4.12,70.0,75.0,1677760970


In [7]:
# Round these off so date time is to the nearest hour.
dfw['datetime'] = pd.to_datetime(dfw['datetime'], unit = 's')
dfw['datetime'] = dfw['datetime'].dt.floor('H')
dfw.head()

dfa['datetime'] = pd.to_datetime(dfw['datetime'], unit = 's')
dfa['datetime'] = dfa['datetime'].dt.floor('H')
dfa.head()

,number,available_bikes,available_bike_stands,status,datetime
0,42,8,22,OPEN,2023-03-02 12:00:00
1,30,2,18,OPEN,2023-03-02 12:00:00
2,54,14,18,OPEN,2023-03-02 12:00:00
3,108,18,17,OPEN,2023-03-02 12:00:00
4,20,22,8,OPEN,2023-03-02 12:00:00


In [8]:
# merge the two dataframes on the common column "key"
df = pd.merge(dfa, dfw, on='datetime')
df.head()

,number,available_bikes,available_bike_stands,status,datetime,main,description,temp,visibility,wind_speed,wind_direction,clouds
0,42,8,22,OPEN,2023-03-02 12:00:00,Clouds,broken clouds,280.72,10000.0,4.12,70.0,75.0
1,42,8,22,OPEN,2023-03-02 12:00:00,Clouds,broken clouds,280.72,10000.0,4.12,70.0,75.0
2,42,8,22,OPEN,2023-03-02 12:00:00,Clouds,broken clouds,280.72,10000.0,4.12,70.0,75.0
3,42,8,22,OPEN,2023-03-02 12:00:00,Clouds,broken clouds,280.95,10000.0,4.12,70.0,75.0
4,42,8,22,OPEN,2023-03-02 12:00:00,Clouds,broken clouds,280.95,10000.0,4.12,70.0,75.0


In [9]:
# Add a column for the day of the week (0=Monday, 6=Sunday)
df['dayofweek'] = df['datetime'].dt.dayofweek

# Change day of the week to weekday or weekend
df['weekday_or_weekend'] = df['dayofweek'].apply(lambda x: 'weekday' if x < 5 else 'weekend')
df = df.drop('dayofweek', axis=1)
# Add a column for the hour of the day (0-23)
df['hour'] = df['datetime'].dt.hour
df = df.drop(['datetime', 'status'], axis = 1)

# saving copy to test features on different models
dfc = df
df.head()

,number,available_bikes,available_bike_stands,main,description,temp,visibility,wind_speed,wind_direction,clouds,weekday_or_weekend,hour
0,42,8,22,Clouds,broken clouds,280.72,10000.0,4.12,70.0,75.0,weekday,12
1,42,8,22,Clouds,broken clouds,280.72,10000.0,4.12,70.0,75.0,weekday,12
2,42,8,22,Clouds,broken clouds,280.72,10000.0,4.12,70.0,75.0,weekday,12
3,42,8,22,Clouds,broken clouds,280.95,10000.0,4.12,70.0,75.0,weekday,12
4,42,8,22,Clouds,broken clouds,280.95,10000.0,4.12,70.0,75.0,weekday,12


In [10]:
df = pd.get_dummies(df, columns=['main', 'description', 'weekday_or_weekend'])
df.head()

,number,available_bikes,available_bike_stands,temp,visibility,wind_speed,wind_direction,clouds,hour,main_Clear,...,description_mist,description_moderate rain,description_overcast clouds,description_ragged shower rain,description_scattered clouds,description_shower rain,description_sleet,description_snow,weekday_or_weekend_weekday,weekday_or_weekend_weekend
0,42,8,22,280.72,10000.0,4.12,70.0,75.0,12,0,...,0,0,0,0,0,0,0,0,1,0
1,42,8,22,280.72,10000.0,4.12,70.0,75.0,12,0,...,0,0,0,0,0,0,0,0,1,0
2,42,8,22,280.72,10000.0,4.12,70.0,75.0,12,0,...,0,0,0,0,0,0,0,0,1,0
3,42,8,22,280.95,10000.0,4.12,70.0,75.0,12,0,...,0,0,0,0,0,0,0,0,1,0
4,42,8,22,280.95,10000.0,4.12,70.0,75.0,12,0,...,0,0,0,0,0,0,0,0,1,0


In [11]:
# Split data into training and testing data
X = df.drop(['available_bikes', 'available_bike_stands'], axis = 1)
y = df['available_bikes']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

### Logistic Regression Model

In [12]:
# create and fit the model
model = LogisticRegression()
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# Calculate root mean squared error (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print("Root Mean Squared Error (RMSE):", rmse)

# Calculate R-squared score
r2 = r2_score(y_test, y_pred)
print("R-squared score:", r2)

print("Mean absolute error:", mean_absolute_error(y_test, y_pred))

Root Mean Squared Error (RMSE): 14.49355661250001
R-squared score: -1.322041993867375
Mean absolute error: 10.979131832797428


c:\Users\nicho\anaconda3\envs\SoftwareEngineering\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Linear Regression Model


In [13]:
# create and fit the model
model = LinearRegression()
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# Calculate root mean squared error (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error (RMSE):", rmse)

# Calculate R-squared score
r2 = r2_score(y_test, y_pred)
print("R-squared score:", r2)

# Calculate mean square error
print("Mean absolute error:", mean_absolute_error(y_test, y_pred))

Root Mean Squared Error (RMSE): 9.505997688090114
R-squared score: 0.0011148979646777857
Mean absolute error: 7.734320338951472


### Random Forest Regressor
 

In [14]:
# Create a random forest regressor with 100 trees
model = RandomForestRegressor(n_estimators = 50, random_state = 36)
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# Calculate root mean squared error (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error (RMSE):", rmse)

# Calculate R-squared score
r2 = r2_score(y_test, y_pred)
print("R-squared score:", r2)

# Calculate mean square error
print("Mean absolute error:", mean_absolute_error(y_test, y_pred))
print(y_pred)

Root Mean Squared Error (RMSE): 1.3747750801132241
R-squared score: 0.9791078358344723
Mean absolute error: 0.414576848874598
[10.   27.    7.92 ...  8.36  7.12  3.18]


### SGD Regressor

In [15]:
# create and fit the model
model = SGDRegressor()
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# Calculate root mean squared error (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error (RMSE):", rmse)

# Calculate R-squared score
r2 = r2_score(y_test, y_pred)
print("R-squared score:", r2)

# Calculate mean square error
print("Mean absolute error:", mean_absolute_error(y_test, y_pred))

Root Mean Squared Error (RMSE): 6545213141206254.0
R-squared score: -4.735520429357067e+29
Mean absolute error: 6447443617111489.0


Getting best results for Random Forest Regression, will test this with different values to see can I improve or get similar results with less features.
### Removing Main weather and description

In [16]:
# Taking copy dfc I save earlier 
df = dfc
# Will test dropping main weather description
df = df.drop(['main', 'description'], axis = 1)
df = pd.get_dummies(df, columns=['weekday_or_weekend'])
X = df.drop(['available_bikes', 'available_bike_stands'], axis = 1)
y = df['available_bikes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

# create and fit the model
model = RandomForestRegressor(n_estimators = 50, random_state = 36)
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# Calculate root mean squared error (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error (RMSE):", rmse)

# Calculate R-squared score
r2 = r2_score(y_test, y_pred)
print("R-squared score:", r2)

# Calculate mean square error
print("Mean absolute error:", mean_absolute_error(y_test, y_pred))

Root Mean Squared Error (RMSE): 1.3161933320982169
R-squared score: 0.9808504091265434
Mean absolute error: 0.3883215434083601


This seems to actually have imporved the model, I will now see what it is like with temperature also removed

In [17]:
# Taking copy dfc I save earlier 
df = dfc
# Will test dropping main weather description
df = df.drop(['main', 'description', 'temp'], axis = 1)
df = pd.get_dummies(df, columns=['weekday_or_weekend'])
X = df.drop(['available_bikes', 'available_bike_stands'], axis = 1)
y = df['available_bikes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

# create and fit the model
model = RandomForestRegressor(n_estimators = 50, random_state = 36)
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# Calculate root mean squared error (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error (RMSE):", rmse)

# Calculate R-squared score
r2 = r2_score(y_test, y_pred)
print("R-squared score:", r2)

# Calculate mean square error
print("Mean absolute error:", mean_absolute_error(y_test, y_pred))

Root Mean Squared Error (RMSE): 1.6208820562106874
R-squared score: 0.9709582383932306
Mean absolute error: 0.48455482848959214


Here we see a an increase in our mean square error so I will keep temp in.
Going to try and remove visibility next

In [18]:
# Taking copy dfc I save earlier 
df = dfc
# Will test dropping main weather description
df = df.drop(['main', 'description', 'visibility'], axis = 1)
df = pd.get_dummies(df, columns=['weekday_or_weekend'])
X = df.drop(['available_bikes', 'available_bike_stands'], axis = 1)
y = df['available_bikes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

# create and fit the model
model = RandomForestRegressor(n_estimators = 50, random_state = 36)
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# Calculate root mean squared error (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error (RMSE):", rmse)

# Calculate R-squared score
r2 = r2_score(y_test, y_pred)
print("R-squared score:", r2)

# Calculate mean square error
print("Mean absolute error:", mean_absolute_error(y_test, y_pred))

Root Mean Squared Error (RMSE): 1.3173852891238766
R-squared score: 0.9808157093232366
Mean absolute error: 0.38932475884244366


Visibility makes practically no difference only 0.003 worse in root mean squared error, will remove as such a small difference and value is nearly always the same, now will test wind speed 

In [19]:
# Taking copy dfc I save earlier 
df = dfc
# Will test dropping main weather description
df = df.drop(['main', 'description', 'visibility', 'wind_speed'], axis = 1)
df = pd.get_dummies(df, columns=['weekday_or_weekend'])
X = df.drop(['available_bikes', 'available_bike_stands'], axis = 1)
y = df['available_bikes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

# create and fit the model
model = RandomForestRegressor(n_estimators = 50, random_state = 36)
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# Calculate root mean squared error (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error (RMSE):", rmse)

# Calculate R-squared score
r2 = r2_score(y_test, y_pred)
print("R-squared score:", r2)

# Calculate mean square error
print("Mean absolute error:", mean_absolute_error(y_test, y_pred))

Root Mean Squared Error (RMSE): 1.352881943209504
R-squared score: 0.9797679482195123
Mean absolute error: 0.37351467615985295


A 0.04 difference in root mean squared error, still quite small but will keep this in as still about 3% difference in prediction accuracy. Now will test wind direction

In [20]:
# Taking copy dfc I save earlier 
df = dfc
# Will test dropping main weather description
df = df.drop(['main', 'description', 'visibility', 'wind_direction'], axis = 1)
df = pd.get_dummies(df, columns=['weekday_or_weekend'])
X = df.drop(['available_bikes', 'available_bike_stands'], axis = 1)
y = df['available_bikes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

# create and fit the model
model = RandomForestRegressor(n_estimators = 50, random_state = 36)
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# Calculate root mean squared error (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error (RMSE):", rmse)

# Calculate R-squared score
r2 = r2_score(y_test, y_pred)
print("R-squared score:", r2)

# Calculate mean square error
print("Mean absolute error:", mean_absolute_error(y_test, y_pred))

Root Mean Squared Error (RMSE): 1.4651151099139612
R-squared score: 0.9762718640596979
Mean absolute error: 0.46396960419716043


Sizeable increase with wind direction gone. Will keep it in.
Finally will check clouds.

In [21]:
# Taking copy dfc I save earlier 
df = dfc
# Will test dropping main weather description
df = df.drop(['main', 'description', 'visibility', 'clouds'], axis = 1)
df = pd.get_dummies(df, columns=['weekday_or_weekend'])
X = df.drop(['available_bikes', 'available_bike_stands'], axis = 1)
y = df['available_bikes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

# create and fit the model
model = RandomForestRegressor(n_estimators = 50, random_state = 36)
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# Calculate root mean squared error (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error (RMSE):", rmse)

# Calculate R-squared score
r2 = r2_score(y_test, y_pred)
print("R-squared score:", r2)

# Calculate mean square error
print("Mean absolute error:", mean_absolute_error(y_test, y_pred))

Root Mean Squared Error (RMSE): 1.3302468789072364
R-squared score: 0.9804392895489352
Mean absolute error: 0.4014508038585209


Small decrease to Mean square error about 0.03. Will leave clouds in. This is out final modal. We have removed main, description, visibility and clouds.

Our features being used in final modal are: 
- Station Number
- Temperature
- Wind Speed
- Wind direction
- Clouds
- Hour
- weekday or weekend

In [22]:
# Taking copy dfc I save earlier 
df = dfc
# Will test dropping main weather description
df = df.drop(['main', 'description', 'visibility'], axis = 1)
df = pd.get_dummies(df, columns=['weekday_or_weekend'])
X = df.drop(['available_bikes', 'available_bike_stands'], axis = 1)
y = df['available_bikes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

model = RandomForestRegressor(n_estimators = 50, random_state = 36)
model.fit(X_train, y_train)
# Serialize model object into a file called model.pkl on disk using pickle
with open('model.pkl', 'wb') as handle:
    pickle.dump(model, handle, pickle.HIGHEST_PROTOCOL)
# pickle.HIGHEST_PROTOCOL using the highest available protocol
# (we used wb to open file as binary and use a higher pickling protocol)


In [23]:
X.head()

,number,temp,wind_speed,wind_direction,clouds,hour,weekday_or_weekend_weekday,weekday_or_weekend_weekend
0,42,280.72,4.12,70.0,75.0,12,1,0
1,42,280.72,4.12,70.0,75.0,12,1,0
2,42,280.72,4.12,70.0,75.0,12,1,0
3,42,280.95,4.12,70.0,75.0,12,1,0
4,42,280.95,4.12,70.0,75.0,12,1,0
